# Some definitions

In [138]:
import requests
import urllib
import json


SERVER = 'http://localhost:3000'

# Create Session

In [139]:
url = urllib.parse.urljoin(SERVER, '/api/session/')

print("HTTP POST {}".format(url))

response = requests.post(url=url, data={"observer_name": "Simon Torres", 
                                        "raw_data_path": "/pipeline/data/raw/NOAO/20190912", 
                                        "reduced_data_path":"/pipeline/data/reduced/NOAO/20190912"})

HTTP POST http://localhost:3000/api/session/


In [140]:
session_id = None
if response.status_code == 200:
    json_response = response.json()
    session_id = json_response['session_id']
    print("Created Session with ID: {}".format(session_id))
else:
    print("Status Code: {}".format(response.status_code))

Created Session with ID: 4


In [141]:
url = urllib.parse.urljoin(SERVER, '/api/session/?id={}'.format(session_id))
print('HTTP GET {}'.format(url))
response = requests.get(url=url)
if response.status_code == 200:
    json_response = response.json()
    print(json.dumps(json_response, indent=4))

HTTP GET http://localhost:3000/api/session/?id=4
{
    "created": "2019-09-26T15:19:01.846107Z",
    "raw_data_path": "/pipeline/data/raw/NOAO/20190912",
    "master_flat": "",
    "reduced_data_path": "/pipeline/data/reduced/NOAO/20190912",
    "trim_section": "",
    "slit_trim_section": "",
    "observer_name": "Simon Torres",
    "overscan_region": "",
    "master_bias": "",
    "session_id": 4
}


# Reduction

The API has to be really "loose" so in this notebook I'll just exemplify the steps.


## Get Overscan and Trim Section/Region

In [142]:
url = urllib.parse.urljoin(SERVER, '/api/session/settings/')

print(url)
response = requests.get(url=url, 
                        params={'id': session_id,
                                'file':'0104_bias_spec1x1.fits'})

print(response.url)
json_response = response.json()
print(json.dumps(json_response, indent=4))

http://localhost:3000/api/session/settings/
http://localhost:3000/api/session/settings/?file=0104_bias_spec1x1.fits&id=4
{
    "overscan_region": "[6:49,1:1896]",
    "trim_section": "[51:4110,2:1896]",
    "session_id": 4
}


In [143]:
response = requests.post(url=url, json=json_response)

json_response = response.json()

print(json.dumps(json_response, indent=4))

{
    "detail": "Successfully updated session 4 settings"
}


Checking the updated status of the session

In [144]:
url = urllib.parse.urljoin(SERVER, '/api/session/')

response = requests.get(url=url, params={'id': session_id})

if response.status_code == 200:
    json_response = response.json()
    print(json.dumps(json_response, indent=4))

{
    "created": "2019-09-26T15:19:01.846107Z",
    "raw_data_path": "/pipeline/data/raw/NOAO/20190912",
    "master_flat": "",
    "reduced_data_path": "/pipeline/data/reduced/NOAO/20190912",
    "trim_section": "[51:4110,2:1896]",
    "slit_trim_section": "",
    "observer_name": "Simon Torres",
    "overscan_region": "[6:49,1:1896]",
    "master_bias": "",
    "session_id": 4
}


## Create Master Bias

In [180]:
url = urllib.parse.urljoin(SERVER, '/api/calibrations/bias/')

response = requests.post(url=url, data={'session_id': session_id, 
                                        'auto_discover': 'true'})

if response.status_code == 200:
    json_response = response.json()
    print(json.dumps(json_response, indent=4))
else:
    print(response.status_code)

{
    "error": "none",
    "master_bias": [
        {
            "master_bias_6": {
                "full_path": "/pipeline/data/reduced/NOAO/20190912",
                "elapsed_time": 10.164152383804321,
                "file_name": "master_bias_6.fits",
                "file_list": [
                    "0100_bias_spec1x1.fits",
                    "0101_bias_spec1x1.fits",
                    "0102_bias_spec1x1.fits",
                    "0103_bias_spec1x1.fits",
                    "0104_bias_spec1x1.fits",
                    "0105_bias_spec1x1.fits",
                    "0106_bias_spec1x1.fits",
                    "0107_bias_spec1x1.fits",
                    "0108_bias_spec1x1.fits",
                    "0109_bias_spec1x1.fits"
                ]
            }
        }
    ]
}


In [146]:
print(response.status_code)

200


In [176]:
master_bias_file = json_response['master_bias']
print(master_bias_file)

[{'master_bias_4': {'full_path': '/pipeline/data/reduced/NOAO/20190912', 'elapsed_time': 90.45678329467773, 'file_name': 'master_bias_4.fits', 'file_list': ['0100_bias_spec1x1.fits', '0101_bias_spec1x1.fits', '0102_bias_spec1x1.fits', '0103_bias_spec1x1.fits', '0104_bias_spec1x1.fits', '0105_bias_spec1x1.fits', '0106_bias_spec1x1.fits', '0107_bias_spec1x1.fits', '0108_bias_spec1x1.fits', '0109_bias_spec1x1.fits']}}]


And for testing, doing a `GET` request to the same url it will retrieve the file name.

In [177]:
response = requests.get(url=url, params={"id": session_id})

print(response.url)

if response.status_code == 200:
    json_response = response.json()
    print(json.dumps(json_response, indent=4))
else:
    print(response.status_code)

http://localhost:3000/api/calibrations/bias/?id=4
{
    "master_bias": [
        {
            "obstype": "BIAS",
            "filter2": "GG455",
            "roi": "Spectroscopic 1x1",
            "session": 4,
            "full_path": "/pipeline/data/reduced/NOAO/20190912",
            "read_noise": "3.89",
            "slit": "1.03\" long slit",
            "file_name": "master_bias.fits",
            "gain": "1.48",
            "cam_targ": "16.1",
            "grating": "SYZY_400",
            "filter": "<NO FILTER>",
            "grt_targ": "7.5"
        },
        {
            "obstype": "BIAS",
            "filter2": "GG455",
            "roi": "Spectroscopic 1x1",
            "session": 4,
            "full_path": "/pipeline/data/reduced/NOAO/20190912",
            "read_noise": "3.89",
            "slit": "1.03\" long slit",
            "file_name": "master_bias_2.fits",
            "gain": "1.48",
            "cam_targ": "16.1",
            "grating": "SYZY_400",
           

In [178]:
response = requests.get(url=url, params={"id": session_id, "file":"0035_dflat_400M2_GG455.fits"})

print(response.url)

if response.status_code == 200:
    json_response = response.json()
    print(json.dumps(json_response, indent=4))
else:
    print(response.status_code)
    
    
master_bias_file = json_response['master_bias']["file_name"]

print(master_bias_file)

http://localhost:3000/api/calibrations/bias/?file=0035_dflat_400M2_GG455.fits&id=4
500


TypeError: list indices must be integers or slices, not str

## Create Master Flats

In [155]:
 url = urllib.parse.urljoin(SERVER, '/api/calibrations/flats/')
    
request_data = {'session_id': session_id, 
                'auto_discover': 'true',
                'saturation_threshold': 1,
                'ignore_bias': 'false',
                'master_bias': master_bias_file}
    
response = requests.post(url=url, data=request_data)

if response.status_code == 200:
    json_response = response.json()
    print(json.dumps(json_response, indent=4))
else:
    print(response.status_code)

{
    "info": "Create master flat",
    "master_flat": [
        {
            "master_flat_400_m2_GG455": {
                "obstype": "FLAT",
                "filter2": "GG455",
                "roi": "Spectroscopic 1x1",
                "session": 4,
                "full_path": "/pipeline/data/reduced/NOAO/20190912",
                "read_noise": 3.89,
                "slit": "1.03\" long slit",
                "file_name": "master_flat_400_m2_GG455.fits",
                "gain": 1.48,
                "cam_targ": 16.1,
                "grating": "SYZY_400",
                "filter": "<NO FILTER>",
                "grt_targ": 7.5
            },
            "file_list": [
                "0035_dflat_400M2_GG455.fits",
                "0036_dflat_400M2_GG455.fits",
                "0037_dflat_400M2_GG455.fits",
                "0038_dflat_400M2_GG455.fits",
                "0039_dflat_400M2_GG455.fits",
                "0040_dflat_400M2_GG455.fits",
                "0041_dflat_400M2_G

In [158]:
response = requests.get(url=url, params={'id': session_id})

if response.status_code == 200:
    json_response = response.json()
    print(json.dumps(json_response, indent=4))
else:
    print(response.status_code)

{
    "master_flat": [
        {
            "obstype": "FLAT",
            "filter2": "GG455",
            "roi": "Spectroscopic 1x1",
            "session": 4,
            "full_path": "/pipeline/data/reduced/NOAO/20190912",
            "read_noise": "3.89",
            "slit": "1.03\" long slit",
            "file_name": "new_master_flat.fits",
            "gain": "1.48",
            "cam_targ": "16.1",
            "grating": "SYZY_400",
            "filter": "<NO FILTER>",
            "grt_targ": "7.5"
        },
        {
            "obstype": "FLAT",
            "filter2": "GG455",
            "roi": "Spectroscopic 1x1",
            "session": 4,
            "full_path": "/pipeline/data/reduced/NOAO/20190912",
            "read_noise": "3.89",
            "slit": "1.03\" long slit",
            "file_name": "master_flat_400_m2_GG455.fits",
            "gain": "1.48",
            "cam_targ": "16.1",
            "grating": "SYZY_400",
            "filter": "<NO FILTER>",
         

## Get Overscan and apply

In [170]:
url = urllib.parse.urljoin(SERVER, '/api/image/overscan/')
    
# request_data = {'session_id': session_id, 
#                 'auto_discover': 'true',
#                 'saturation_threshold': 1,
#                 'ignore_bias': 'false',
#                 'master_bias': master_bias_file}
    
response = requests.get(url=url, params={'id': session_id, 'file': '0073_CVSO_200.fits'})

json_response = response.json()
if response.status_code == 200:
    
    print(json.dumps(json_response, indent=4))
else:
    
    print("{}: {}".format(response.status_code, json_response))
overscan_region = json_response["overscan_region"]
print(overscan_region)


{
    "overscan_region": "[6:49,1:1896]"
}
[6:49,1:1896]


In [171]:
request_data = {'session_id': session_id, 
                'overscan_region': overscan_region,
                'file': '0073_CVSO_200.fits'}
    
response = requests.post(url=url, data=request_data)

json_response = response.json()
if response.status_code == 200:
    
    print(json.dumps(json_response, indent=4))
else:
    
    print("{}: {}".format(response.status_code, json_response))


{
    "overscan_region": "[6:49,1:1896]",
    "overscan_corrected": "o_0073_CVSO_200.fits",
    "original_file": "0073_CVSO_200.fits"
}


## Get trim section

In [172]:
url = urllib.parse.urljoin(SERVER, '/api/image/trim/')
    
# request_data = {'session_id': session_id, 
#                 'auto_discover': 'true',
#                 'saturation_threshold': 1,
#                 'ignore_bias': 'false',
#                 'master_bias': master_bias_file}
    
response = requests.get(url=url, params={'id': session_id, 
                                         'file': 'o_0073_CVSO_200.fits', 
                                         'path_type': 'reduced'})

json_response = response.json()
if response.status_code == 200:
    
    print(json.dumps(json_response, indent=4))
else:
    
    print("{}: {}".format(response.status_code, json_response))
trim_section = json_response["trim_section"]
print(trim_section)


{
    "trim_section": "[51:4110,2:1896]"
}
[51:4110,2:1896]
